# Arize Tutorial: Model Type - Score Categorical

Let's get started on using Arize!✨

Arize helps you visualize your model performance, understand drift & data quality issues, and share insights learned from your models.

In this tutorial, we will be building a model to predict if someone has breast cancer or not. The model predicts a score (how likely is this person to have breast cancer) and based on a threshold, determines the class (True/False). For this model, we will log the score and the class so this model type is `ModelType.SCORE_CATEGORICAL`. We will load the models’s training, validation, and test inferences into Arize. 🚀.

Finally, using the `arize.pandas.logger` (available in `arize 2.2.0+`), we will load the model's inference data into Arize. 🚀

### Running This Notebook
1. Save a copy in Google Drive for yourself.
2. Step through each section below, pressing play on the code blocks to run the cells.
3. In Step 2, use your own Space and API key from your Arize account.


## Step 1: Load Data and Build Model

In [1]:
!pip install -q arize shap

import datetime
import uuid
import shap

import numpy as np
import pandas as pd
from arize.pandas.logger import Client, Schema
from arize.utils.types import ModelTypes, Environments

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
# 1 Load data and split data
data = datasets.load_breast_cancer()
X, y = datasets.load_breast_cancer(return_X_y=True)

# NOTE: We need to set y.astype(str) since BINARY expected non-integer.
X, y = X.astype(np.float32), y.astype(str)
X, y = pd.DataFrame(X, columns=data["feature_names"]), pd.Series(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42)

# 2 Fit a simple logistic regression model
clf = LogisticRegression(max_iter=3000, verbose=False).fit(X_train, y_train)

# 3 Use the model to generate predictions
y_train_pred = clf.predict(X_train)
y_train_pred_proba = clf.predict_proba(X_train)

y_val_pred = clf.predict(X_val)
y_val_pred_proba = clf.predict_proba(X_val)

y_test_pred = clf.predict(X_test)
y_test_pred_proba = clf.predict_proba(X_test)

# 4 Map the prediction probability to prediction score
def map_proba(y_pred, y_pred_proba):
    """
    Input:
    y_pred (1-dim) and y_pred_proba (n-dim) from sklearn
    Output:
    y_pred_scores (1-dim) np.array for the probability of only the predicted class
    """
    y_pred_scores = [y_pred_proba[i][int(y_pred[i])] for i in range(len(y_pred))]
    return pd.Series(y_pred_scores)


y_train_pred_score = map_proba(y_train_pred, y_train_pred_proba)
y_val_pred_score = map_proba(y_val_pred, y_val_pred_proba)
y_test_pred_score = map_proba(y_test_pred, y_test_pred_proba)

print("Step 1 ✅: Load Data & Build Model Done!")

Step 1 ✅: Load Data & Build Model Done!


Group and combine all the data (features/predictions/actuals) for each environment (train/validation/test) into one pd.DataFrame object.

In [3]:
# df for training env
train_df = X_train.reset_index(drop=True)
train_df["prediction_label"] = y_train_pred
train_df["prediction_score"] = y_train_pred_score
train_df["actual_label"] = list(y_train)
train_df["prediction_id"] = [str(uuid.uuid4()) for _ in range(len(y_train))]


# df for validation env
val_df = X_val.reset_index(drop=True)
val_df["prediction_label"] = y_val_pred
val_df["prediction_score"] = y_val_pred_score
val_df["actual_label"] = list(y_val)
val_df["prediction_id"] = [str(uuid.uuid4()) for _ in range(len(y_val))]


# df for production env
test_df = X_test.reset_index(drop=True)
test_df["prediction_label"] = y_test_pred
test_df["prediction_score"] = y_test_pred_score
test_df["actual_label"] = list(y_test)
test_df["prediction_id"] = [str(uuid.uuid4()) for _ in range(len(y_test))]

# simulate predictions evenly distributed over 30 days by manually specifying prediction time
current_time = datetime.datetime.now().timestamp()
earlier_time = (datetime.datetime.now() - datetime.timedelta(days=30)).timestamp()
optional_prediction_timestamps = np.linspace(
    earlier_time, current_time, num=len(y_test)
)
test_df["prediction_ts"] = pd.Series(optional_prediction_timestamps.astype(int))

**Optional** - add SHAP value to the simulated production environment data to enable the model explainability feature on Arize platform

**SHAP (SHapley Additive exPlanations)** is a game theoretic approach to explain the output of any machine learning model.

For more in-depth usage of the `shap` library, visit [SHAP Core Explainers](https://shap-lrjball.readthedocs.io/en/docs_update/generated/shap.Explainer.html) and pick an explainer specific to your machine learning model. `shap.Explainer` is the default explainer that will match model type, but you can specify your own type. For example, you can choose to use for example `shap.TreeExplainer`, but it won't work on models such as `sklearn.LinearModel.LogisticRegression`.

In [4]:
def get_shap_values(model, X_data, ExplainerType=shap.Explainer, show_graph=False):
    # NOTE: If there are errors, you  need to manually choose which explainer class
    explainer = ExplainerType(model, X_data)
    shap_values = explainer.shap_values(X_data)
    # When not in production, it can be helpful to check graphs for feature explainability
    if show_graph:
        shap.summary_plot(shap_values, X_data, feature_names=data["feature_names"])

    return pd.DataFrame(
        shap_values, columns=[f"{fn}_shap" for fn in data["feature_names"]]
    )


test_shap_df = get_shap_values(clf, X_test)
test_df = pd.concat([test_df, test_shap_df], axis=1)

## Step 2: Import and Setup Arize Client
You can find your `API_KEY` and `SPACE_KEY` by navigating to the settings page in your workspace (only space admins can see the keys). Copy those over to the set-up section. We will also be setting up some metadata to use across all logging.
<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [5]:
SPACE_KEY = 'SPACE_KEY'
API_KEY = 'API_KEY'
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

model_id = "breast_cancer_score_prediction"
model_version = "1.0"

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("Step 2 ✅: Import and Setup Arize Client Done! Now we can start using Arize!")

Step 2 ✅: Import and Setup Arize Client Done! Now we can start using Arize!


## Step 3: Log Inferences to Arize with arize.pandas.logger

### 3.1: Log training data to Arize

In [6]:
# Define a Schema() for Arize to pick up the data from the correct column for logging
train_schema = Schema(
    prediction_id_column_name="prediction_id",
    prediction_label_column_name="prediction_label",
    prediction_score_column_name="prediction_score",
    actual_label_column_name="actual_label",
    feature_column_names=train_df.columns.drop(
        ["prediction_id", "prediction_label", "prediction_score", "actual_label"]
    ),
)

train_res = arize_client.log(
    dataframe=train_df,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.TRAINING,
    schema=train_schema,
)
if train_res.status_code != 200:
    print(f"❌ future failed with response code {train_res.status_code}, {train_res.text}")
else:
    print(f"✅ future completed with response code {train_res.status_code}")

future completed with response code 200


### 3.2: Log validation data to Arize

In [7]:
val_schema = Schema(
    prediction_id_column_name="prediction_id",
    prediction_label_column_name="prediction_label",
    prediction_score_column_name="prediction_score",
    actual_label_column_name="actual_label",
    feature_column_names=val_df.columns.drop(
        ["prediction_id", "prediction_label", "prediction_score", "actual_label"]
    ),
)

val_res = arize_client.log(
    dataframe=val_df,
    model_id=model_id,
    model_version=model_version,
    batch_id="validation_test",  # provide a batch_id to distinguish from other validation data set
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.VALIDATION,
    schema=val_schema,
)
if val_res.status_code != 200:
    print(f"❌ future failed with response code {val_res.status_code}, {val_res.text}")
else:
    print(f"✅ future completed with response code {val_res.status_code}")

future completed with response code 200


### 3.3: Log production data to Arize
Note: We will be sending the test data to emulate sending production data.

In [8]:
# Logging production
all_cols = test_df.columns
shap_cols = test_shap_df.columns
feature_cols = all_cols.drop(
    list(shap_cols)
    + [
        "prediction_id",
        "prediction_ts",
        "prediction_label",
        "prediction_score",
        "actual_label",
    ]
)

test_schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="prediction_label",
    prediction_score_column_name="prediction_score",
    actual_label_column_name="actual_label",
    feature_column_names=feature_cols,
    shap_values_column_names=dict(zip(feature_cols, shap_cols)),
)

test_res = arize_client.log(
    dataframe=test_df,
    model_id=model_id,
    model_version=model_version,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.PRODUCTION,
    schema=test_schema,
)
if test_res.status_code != 200:
    print(f"❌ future failed with response code {test_res.status_code}, {test_res.text}")
else:
    print(f"✅ future completed with response code {test_res.status_code}")

future completed with response code 200


### Check Data Ingestion Information

Data will be available in the UI in about 10 minutes after it was received. If data from a new model is sent, the model will be reflected almost immediately in the Arize platform. However, you will not see data yet. To verify data has been sent correctly and is being processed, we recommend that you check our Data Ingestion tab.

You will be able to see the predictions, actuals, and feature importances that have been sent in the last week, last day or last 30 minutes.

An example view of the Data Ingestion tab from a model, when data is sent continuously over 30 minutes, is shown in the image below.

<img src="https://storage.cloud.google.com/arize-assets/fixtures/data-ingestion-tab.png" width="700">



# Arize Overview
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.